In [1]:
import yaml
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#import scripts files
from scripts.vision import Vision
from scripts.global_nav import GlobalNav
from scripts.local_nav import LocalNav
from utils.utils import *

In [6]:
# Load the configuration file
with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Controller configuration
angle_tolerance = config['controller']['angle_tolerance']
distance_tolerance = config['controller']['distance_tolerance']
scale_rotation_speed = config['controller']['scale_rotation_speed']
max_rotation_speed = config['controller']['max_rotation_speed']
scale_translation_speed = config['controller']['scale_translation_speed']
min_translation_speed = config['controller']['min_translation_speed']
max_translation_speed = config['controller']['max_translation_speed']

# World configuration
world_width = config['world']['width']
world_height = config['world']['height']

# Thymio configuration
thymio_width = config['thymio']['size']['width']
thymio_length = config['thymio']['size']['length']

# Webcam configuration
webcam_device_id = config['webcam']['device_id']
webcam_matrix = np.array(config['webcam']['matrix'])
webcam_distortion = np.array(config['webcam']['distortion'])
webcam_resolution = config['webcam']['resolution']
webcam_padding = config['webcam']['padding']


In [7]:
vision = Vision(
    device_id=webcam_device_id,
    camera_matrix=webcam_matrix,
    dist_coeffs=webcam_distortion,
    resolution=webcam_resolution,
    padding=webcam_padding,
    world_width=world_width,
    world_height=world_height
)

Vision Initialized


In [8]:
# Try connecting to the webcam
print(f"Trying to connect to device {webcam_device_id}...")
if not vision.connect_webcam():
    print("Could not find webcam on any device ID. Please check connection.")
    exit(1)
else: 
    print(f"Successfully connected to device {webcam_device_id}")

Trying to connect to device 0...
Successfully connected to device 0


In [12]:
while True:
    frame, process_frame, thymio_position, goal_position = vision.get_frame()
    
    original_view = frame.copy()
    if process_frame is not None:
        process_view = process_frame.copy()
    trajectory_view = frame.copy()  # You can add trajectory visualization here
    final_view = frame.copy()  # You can add final processing here
    
    # Add labels to each view
    original_view = add_label(original_view, "Original View")
    process_view = add_label(original_view, "Process View")
    trajectory_view = add_label(trajectory_view, "Trajectory View")
    final_view = add_label(final_view, "Final View")
    
    # Calculate dimensions for the combined frame
    height = frame.shape[0]
    width = frame.shape[1]
    
    # Create a blank canvas for 2x2 grid
    combined_frame = np.zeros((height * 2, width * 2, 3), dtype=np.uint8)
    
    # Place all four views in 2x2 grid
    # Top left
    combined_frame[0:height, 0:width] = original_view
    # Top right
    combined_frame[0:height, width:] = process_view
    # Bottom left
    combined_frame[height:, 0:width] = trajectory_view
    # Bottom right
    combined_frame[height:, width:] = final_view
    
    # Show the combined frame
    cv2.imshow('Quad Stream', combined_frame)

vision.cleanup_webcam()

: 

In [23]:


def add_label(image, text):
    # Add black background for text
    cv2.rectangle(image, (0, 0), (200, 30), (0, 0, 0), -1)
    # Add white text
    cv2.putText(image, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (255, 255, 255), 2)
    return image

while True:
    # Read frame from camera
    ret, frame = camera.read()
    if not ret:
        break
    
    # Create copies for different views
    original_view = frame.copy()
    process_view = frame.copy()  # You can add processing here
    trajectory_view = frame.copy()  # You can add trajectory visualization here
    final_view = frame.copy()  # You can add final processing here
    
    # Add labels to each view
    original_view = add_label(original_view, "Original View")
    process_view = add_label(process_view, "Process View")
    trajectory_view = add_label(trajectory_view, "Trajectory View")
    final_view = add_label(final_view, "Final View")
    
    # Calculate dimensions for the combined frame
    height = frame.shape[0]
    width = frame.shape[1]
    
    # Create a blank canvas for 2x2 grid
    combined_frame = np.zeros((height * 2, width * 2, 3), dtype=np.uint8)
    
    # Place all four views in 2x2 grid
    # Top left
    combined_frame[0:height, 0:width] = original_view
    # Top right
    combined_frame[0:height, width:] = process_view
    # Bottom left
    combined_frame[height:, 0:width] = trajectory_view
    # Bottom right
    combined_frame[height:, width:] = final_view
    
    # Show the combined frame
    cv2.imshow('Quad Stream', combined_frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [7]:
camera.release()